# Batch Normalization

paper: https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/43442.pdf

## 문제상황

SGD는 간단하면서도 효율적이었으나, learning rate 등의 hyper parameter의 세심한 튜닝이 필요. <br>
모델 layer가 깊어질수록 model parameter의 작은 변화가 다음 layer에 큰 영향을 줄 수 있음. <br>

**Internal Covariance Shift** <br>
학습 중 parameter에 의해 network 각 노드의 입력 분포가 변화는 것을 내부 공변량 변화라고 부름. <br>
(※ covariate shift: train과 test data의 분포가 다른 것) <br>
layer의 입력 분포가 변경되면 layer가 새로운 분포를 지속적으로 학습해야 하는 부담 존재. <br>

**Whitening** <br>
입력 feature를 uncorrelated 하게 만들며, 분산을 평균 0, 표준편차 1인 입력값으로 정규화 시키는 방법 <br>
optimizatino step에서 이러한 변화가 누적되면, parameter update를 정규화를 업데이트하는 방식으로 할 수 있는 위험이 존재 <br>
b의 업데이트와 그에 따른 정규화의 변화가 레이어의 출력이나 결과적으로 손실에 영향을 주지 못하는 경우가 존재 <br>
특히나 정규화가 scale을 조절하게 되면, 이러한 문제가 더욱 심화

ex)

Assume $ x = u + b $, u: layer input, b: bias

normalization: $ \bar{x} = x - E[x] $, where \( E[x] \)는 training set에서 \( x \)의 평균

가정: gradient descent의 한 스텝에서 \( b \)의 업데이트가 \( E[x] \)에 미치는 영향을 무시 <br>
이 경우 \( b \)는 다음과 같이 update: <br>
$ b \leftarrow b + \Delta b $ <br>
여기서 $ \Delta b $는 loss의 gradient와 연관, where $ \Delta b \propto -\frac{\partial \ell}{\partial \bar{x}}$ <br>

그러면 새로운 입력 \( x' \)는 다음과 같이 표현: <br>
$ x' = u + (b + \Delta b) $

이제 정규화 식을 적용하면 아래와 같음: <br>
$ \bar{x}' = x' - E[x'] $

이때, \( E[x'] \)는 \( b \)의 변화에 따라 변하는데, 만약 \( E[x'] \)의 변화가 무시된다면, 아래와 같은 식을 얻을 수 있음 <br>
$ u + (b + \Delta b) - E[u + (b + \Delta b)] = u + b - E[u + b] $ <br>

결국, 이로 인해 출력이나 손실에 변화가 없게 됨

## Batch Normalization이란?

|||
|-|-|
|<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FPYpzO%2FbtqEbvPCvsc%2F3x9sukTLAwdqNWOkpwgTAk%2Fimg.png" width="500" height="300"/>|<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdrYZz3%2FbtqEcnDzpsJ%2F2j0nd5KoikKjx1ZhL2rEKK%2Fimg.png" width="500" height="300"/>|

<br>

각 레이어의 입력을 완전히 화이트닝하는 것은 비용이 많이 들고 모든 곳에서 미분 가능하지 않기 때문에, 두 가지 간소화 적용
1. 레이어 입력과 출력을 함께 화이트닝하는 대신, 각 스칼라 피쳐를 독립적으로 정규화 <br>
    각 $x^{(k)}$ 의 평균을 0으로, 분산을 1로 만듦

    $$
    \bar{x}^{(k)} = \frac{x^{(k)} - E[x^{(k)}]}{\sqrt{\text{Var}[x^{(k)}]}}
    $$

    여기서 기대값 $ E[x^{(k)}] $ 과 분산 $ \text{Var}[x^{(k)}] $는 훈련 데이터 세트에 대해 계산 <br>
    -> 피쳐가 서로 상관관계가 없어도 수렴 속도를 높이는 데 도움 <br>
    다만, 각 입력을 단순히 정규화하는 것은 레이어가 표현할 수 있는 것을 변경할 수 있음 <br>
    ex) 시그모이드 함수의 입력을 정규화하면 입력이 비선형 함수의 선형 영역으로 제한될 수 있음 <br>
    이를 해결하기 위해, 각 활성화 $ x^{(k)} $에 대해 두 개의 파라미터 $ \gamma^{(k)} $와 $ \beta^{(k)} $를 도입하여 정규화된 값을 스케일하고 이동

    $$
    y^{(k)} = \gamma^{(k)} \bar{x}^{(k)} + \beta^{(k)}
    $$

    이 파라미터들은 원래 모델 파라미터와 함께 학습되며, 네트워크의 표현력을 복원 <br>
    $ \gamma^{(k)} = \sqrt{\text{Var}[x^{(k)}]} $와 $ \beta^{(k)} = E[x^{(k)}] $로 설정하면 원래 활성화 함수의 결과 복원
2. 미니 배치를 사용할 때 각 활성화의 평균과 분산을 추정 <br>
    -> 정규화에 사용되는 통계량이 경량 하강법의 역전파에 완전히 참여 <br>
   미니 배치의 크기가 화이트닝되는 활성화 수보다 작을 가능성이 높기 때문에, 차원별 분산을 계산하는 것이 가능하고, 이는 합동 공분산을 계산할 때 필요한 정규화가 필요 없음을 의미

   미니 배치 $ B $의 크기를 $ m $이라고 할 때, 각 활성화는 독립적으로 정규화 <br>
   특정 활성화 $ x^{(k)} $에 대해, 미니 배치는 아래와 같음:

   $$
   B = \{ x_1, x_2, \ldots, x_m \}
   $$

   정규화된 값을 $ x_1, \ldots, x_m $라고 하고, 이들의 선형 변환을 $ y_1, \ldots, y_m $라고 하며, 배치 정규화 변환(Batch Normalizing Transform)이라고 칭함.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcFYkLE%2FbtqEcUnlXKy%2FZbGZNjObjo2gL2xss8zYzk%2Fimg.png" width="400" height="400"/>

$ \gamma $와 $ \beta $는 아래의 수식에 의해 update

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F4RkdF%2FbtqEdkTEWnl%2FCmZoRyLKCa9LhrGsPgJUa0%2Fimg.png" width="400" height="300"/>

## 학습과 추론 방식의 차이

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdBNbiQ%2FbtqEdwM6VCc%2FcoJrE7fDA7k132eZZJ5Ss0%2Fimg.png" width="400" height="300"/>

training: 미니 배치 크기 $m > 1$을 가진 optimizer를 통해 training <br>
inference: 추론 시 출력은 입력에만 결정적으로 의존해야 하므로, 훈련 후에는 다음과 같은 정규화를 사용 <br>
$$ \bar{x} = \frac{x - E[x]}{\sqrt{\text{Var}[x] + \epsilon}} $$

분산은 아래의 추정량 사용 <br>

$$
\text{Var}[x] = \frac{m}{m-1} \cdot E_B[\sigma^2_B] \\
\text{where } \sigma_B^2: \text{training mini-batch sample variance}
$$

추론 중 평균과 분산이 고정되므로, 정규화는 각 활성화에 대해 단순한 선형 변환으로 적용. <br>
추가적으로, 스케일링 𝛾와 이동 𝛽가 함께 적용되어 BN($x$)를 대체하는 단일 선형 변환을 생성.


<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcsUzfW%2FbtqEdwzAYcD%2FYtVk1gkzwqjDhiBhb0spMk%2Fimg.png" width="400" height="300"/>

## 사용 방법

일반적인 사용법은 layer output 이후 activation 이전에 위치<br>
최종 출력에는 batch normalization을 적용하지 않음 <br>

<br>

> ```python
> class Model(nn.Module):
>     def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
>         super().__init__()
>         self.input_dim = input_dim
>         self.hidden_dim = hidden_dim
>         self.output_dim = output_dim
> 
>         self.linear1 = nn.Linear(input_dim, hidden_dim)
>         self.batch_normalization1 = nn.BatchNorm1d(hidden_dim)  # 첫 번째 배치 정규화
>         self.relu1 = nn.ReLU()
>         self.dropout1 = nn.Dropout(dropout_prob)
>         
>         self.linear2 = nn.Linear(hidden_dim, hidden_dim)
>         self.batch_normalization2 = nn.BatchNorm1d(hidden_dim)  # 두 번째 배치 정규화
>         self.relu2 = nn.ReLU()
>         self.dropout2 = nn.Dropout(dropout_prob)
>         
>         self.output = nn.Linear(hidden_dim, output_dim)
> 
>     def forward(self, x):
>         x = self.linear1(x)
>         x = self.batch_normalization1(x)  # 첫 번째 배치 정규화 적용
>         x = self.relu1(x)
>         x = self.dropout1(x)
>         
>         x = self.linear2(x)
>         x = self.batch_normalization2(x)   # 두 번째 배치 정규화 적용
>         x = self.relu2(x)
>         x = self.dropout2(x)
>         
>         x = self.output(x)
>
>         return x
> ```